In [28]:
def preprocess(path="./kakao_data/sink.txt"):
    with open(path, "r") as f:
            lines = f.readlines()
    output = []
    current_paragraph = ""
    for line in lines:
        if line.startswith('#'):
            if current_paragraph:
                output.append(current_paragraph.strip().replace('#',''))
            current_paragraph = line.strip()
        else:
            current_paragraph += ' ' + line.strip()
    if current_paragraph:
        output.append(current_paragraph.strip()[:])

    ret = [output[0] + doc for doc in output[1:]]
    return ret

output = preprocess()

In [29]:
output

['카카오싱크:시작하기 이 문서는 카카오싱크에 대해 소개하고, 카카오싱크 도입에 필요한 검수 및 설정에 대해 안내합니다.',
 '카카오싱크:기능 소개 카카오싱크는 소셜 로그인인 카카오 로그인을 통해 보다 편리하게 서비스에 가입할 수 있도록 도와주는 비즈니스 설루션입니다. 카카오싱크가 제공하는 핵심 기능은 다음 두 가지입니다.  기능 | 설명 | 효과 간편가입 | 동의 화면에서 서비스 약관까지 한 번에 동의받을 수 있습니다. | 서비스 약관 동의 절차 생략 가능 더 다양한 사용자 정보 활용 | 서비스 회원 가입 시 필요한 다양한 사용자 정보를 제공받을 수 있습니다. 이름, 이메일, 전화번호, 연령대, 생일, 성별, 출생연도, 배송지 등 정보를 제공합니다. | 회원 정보 입력 절차 생략 가능',
 '카카오싱크: 과정 예시 1. 카카오로 시작하기 버튼 : 사용자가 카카오싱크 도입 서비스에서 [카카오로 시작하기] 버튼을 눌러 로그인을 요청합니다. 카카오싱크 도입 서비스에서 사용자는 서비스 회원 ID와 비밀번호 입력 대신 카카오톡을 통해 손쉽게 로그인할 수 있습니다. 카카오톡 실행이 불가능한 환경이라도 카카오계정을 사용해 별도 서비스 회원 가입 절차 없이 로그인할 수 있습니다.  2. 동의하고 계속하기 : 사용자는 카카오톡 또는 카카오계정으로 로그인한 후, 동의 화면에서 정보 제공 동의 항목과 서비스 약관 모두 한 번에 동의할 수 있습니다. 카카오싱크 도입 서비스의 동의 화면은 서비스 약관 동의를 포함해, 한 화면에서 회원 가입에 필요한 동의를 모두 받을 수 있도록 지원합니다. 사용자는 동의 화면에서 서비스의 카카오톡 채널을 친구로 추가할 수도 있습니다.  3. 반갑습니다! 회원이 되신 것을 환영합니다.(문구) : 서비스는 카카오에 로그인한 사용자의 사용자 정보 제공을 요청합니다. 카카오싱크 도입 서비스는 일반적인 회원 가입 시 필요한 사용자 정보들을 카카오로부터 제공받을 수 있습니다. 서비스는 제공받은 사용자 정보로 별도 회원 정보 입력 과정을 거치지 않고 즉시 회원

In [30]:
preprocessed_sink = preprocess("./kakao_data/sink.txt")
preprocessed_channel = preprocess("./kakao_data/channel.txt")
preprocessed_social = preprocess("./kakao_data/social.txt")

In [31]:
all_doc = preprocessed_sink + preprocessed_channel + preprocessed_social

In [35]:
import numpy as np

In [37]:
chunk_size = np.median([len(doc) for doc in all_doc])

In [38]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter # text chunk 역할
from langchain.vectorstores import Chroma

In [60]:
CHROMA_PERSIST_DIR = "../chroma-persist"
CHROMA_COLLECTION_NAME = "dosu-bot"

In [61]:
import os
import openai

In [62]:
# local에 있는 api key 읽기
def read_env():
    with open("../.env", "r") as f:
        return f.read().strip()
    
openai.api_key = read_env()
os.environ["OPENAI_API_KEY"] = openai.api_key

In [64]:
for doc in all_doc:
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=100)
    splited_doc = text_splitter.split_text(doc)

    Chroma.from_texts(
        splited_doc,
        OpenAIEmbeddings(),
        collection_name=CHROMA_COLLECTION_NAME,
        persist_directory=CHROMA_PERSIST_DIR,
    )
print('db success')

db success


In [65]:
from pprint import pprint

db = Chroma(
    persist_directory=CHROMA_PERSIST_DIR,
    embedding_function=OpenAIEmbeddings(),
    collection_name=CHROMA_COLLECTION_NAME,
)

In [72]:
docs = db.similarity_search("카카오 소셜 이용 정책은 어떻게 해?")
pprint(docs)

[Document(page_content='카카오 소셜:이해하기 이 문서는 카카오톡 소셜 API를 소개합니다.'),
 Document(page_content='카카오 소셜:기능 소개 카카오톡 소셜 API는 서비스의 소셜 기능 구현을 위한 사용자의 카카오톡 프로필과 친구 정보를 제공합니다. 제공받은 카카오톡 친구 정보를 토대로 카카오톡 메시지를 보낼 수도 있습니다.'),
 Document(page_content='카카오싱크:기능 소개 카카오싱크는 소셜 로그인인 카카오 로그인을 통해 보다 편리하게 서비스에 가입할 수 있도록 도와주는 비즈니스 설루션입니다. 카카오싱크가 제공하는 핵심 기능은 다음 두 가지입니다.  기능 | 설명 | 효과 간편가입 | 동의 화면에서 서비스 약관까지 한 번에 동의받을 수 있습니다. | 서비스 약관 동의 절차 생략 가능 더 다양한 사용자 정보 활용 | 서비스 회원 가입 시 필요한 다양한 사용자 정보를 제공받을 수 있습니다. 이름, 이메일, 전화번호, 연령대, 생일, 성별, 출생연도, 배송지 등 정보를 제공합니다. | 회원 정보 입력 절차 생략 가능'),
 Document(page_content='카카오 소셜:#이용 정책  1.사용 권한 신청 피커와 친구 목록 가져오기 API는 사용 권한이 주어진 앱에서만 사용할 수 있습니다. 사용 권한 신청 방법은 아래와 같습니다.  -피커 또는 친구 목록 가져오기 API를 사용한 소셜 기능 구현 -권한을 받기 전에도 개발 및 테스트를 위해 팀원에 한해 기능 동작 -앱의 팀원뿐 아니라 일반 사용자를 대상으로 친구 정보를 받으려면 권한 필요 -앱의 팀원을 대상으로 기능이 정상 동작하는지 확인 -데브톡에서 [새글 쓰기] 클릭 후, [친구 API 사용 신청] 카테고리를 선택하여 신청 양식 작성  카카오 플랫폼 관리자가 신청 내용을 확인한 뒤 친구 목록 가져오기 API 사용 권한 제공 여부를 답변합니다. 검수를 거쳐 사용 권한을 받으면 서비스의 모든 사용자가 친구 목록 가져오기 API 기능을 이용할 수 있습니다.